In [66]:
# step-1 Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
  # import the evaluation metrics used to measure the accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score
  # import the model_selection to divide the data set into training set and testing set
from sklearn.model_selection import train_test_split
  # Importing the library for preprocessing and hyberparameter tuning
from sklearn.preprocessing import StandardScaler
  # importing the machine learning model we are using random forest regressor
from sklearn.ensemble import RandomForestRegressor

In [67]:
# step-2 Load the Data set
 # Assigning the data set name to a variable name
dataset='SupplyChainEmissionFactorsforUSIndustriesCommodities.xlsx'
 # Dividing the years from 2010 to 2017
years=range(2010,2017)
 # Loading the dataset by using the read_excel file for commodities and industries
commodities_data=pd.read_excel(dataset,sheet_name=f'{years[0]}_Detail_Commodity')
industries_data=pd.read_excel(dataset,sheet_name=f'{years[0]}_Detail_Industry')


In [68]:
# Checking the Commodities data are correctly loaded or not by using df.head() method
commodities_data.head(2)

,Commodity Code,Commodity Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",carbon dioxide,"kg/2018 USD, purchaser price",0.398,0.073,0.470,NaN,4,3,1,4,1
1,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",methane,"kg/2018 USD, purchaser price",0.001,0.001,0.002,NaN,4,3,1,1,1


In [69]:
industries_data.head(2)

,Industry Code,Industry Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,1111A0,Oilseed farming,carbon dioxide,"kg/2018 USD, purchaser price",0.414,0.073,0.487,NaN,4,3,1,4,1
1,1111A0,Oilseed farming,methane,"kg/2018 USD, purchaser price",0.001,0.001,0.002,NaN,4,3,1,1,1


In [70]:
# Step-3 Data Preprocessing
years_data=[]
for i in years:
    try:
        # Dividing the given dataset into different sub datasets based on the year of each industry and commodity
        df_commodity=pd.read_excel(dataset,sheet_name=f'{i}_Detail_Commodity')
        df_industry=pd.read_excel(dataset,sheet_name=f'{i}_Detail_Industry')
        #Adding source and year columns to the newely created datsets
        df_commodity['Source']='Commodity'
        df_industry['Source']='Industry'
        df_commodity['Year']=df_industry['Year']=i
        #  Remove white spaces form column names
        df_commodity.columns=df_commodity.columns.str.strip()
        df_industry.columns=df_industry.columns.str.strip()
        # Rename Columns for consistency
        df_commodity.rename(columns={'Commodity_Code':'Code','Commodity_Name':'Name'},inplace=True)
        df_industry.rename(columns={'Industry_Code':'Code','Industy_Name':'Name'},inplace=True)
        # Concatenate commodity and industry data and append to the list
        years_data.append(pd.concat([df_commodity, df_industry],ignore_index=True))
    except Exception as e:
        print("Error Processing Year:", i, "Error:", str(e))

In [71]:
# Combine all the yaers data into a single DataFrame
df=pd.concat(years_data,ignore_index=True)
df.head(1)

,Commodity Code,Commodity Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins,Source,Year,Industry Code,Industry Name
0,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",carbon dioxide,"kg/2018 USD, purchaser price",0.398,0.073,0.47,NaN,4,3,1,4,1,Commodity,2010,NaN,NaN


In [76]:
df.columns
#  printing the name of the columns

Index(['Commodity Code', 'Commodity Name', 'Substance', 'Unit',
       'Supply Chain Emission Factors without Margins',
       'Margins of Supply Chain Emission Factors',
       'Supply Chain Emission Factors with Margins',
       'DQ ReliabilityScore of Factors without Margins',
       'DQ TemporalCorrelation of Factors without Margins',
       'DQ GeographicalCorrelation of Factors without Margins',
       'DQ TechnologicalCorrelation of Factors without Margins',
       'DQ DataCollection of Factors without Margins', 'Source', 'Year',
       'Industry Code', 'Industry Name'],
      dtype='object')

In [77]:
df.isnull().sum()

Commodity Code                                            11060
Commodity Name                                            11060
Substance                                                     0
Unit                                                          0
Supply Chain Emission Factors without Margins                 0
Margins of Supply Chain Emission Factors                      0
Supply Chain Emission Factors with Margins                    0
DQ ReliabilityScore of Factors without Margins                0
DQ TemporalCorrelation of Factors without Margins             0
DQ GeographicalCorrelation of Factors without Margins         0
DQ TechnologicalCorrelation of Factors without Margins        0
DQ DataCollection of Factors without Margins                  0
Source                                                        0
Year                                                          0
Industry Code                                             11032
Industry Name                           

In [ ]:
#  Here, we are dropping the unnamed column and renaming the columns to match the expected output.
df.drop(columns=['Unnamed: 7'],inplace=True)

In [75]:
df.isnull().sum()

Commodity Code                                            11060
Commodity Name                                            11060
Substance                                                     0
Unit                                                          0
Supply Chain Emission Factors without Margins                 0
Margins of Supply Chain Emission Factors                      0
Supply Chain Emission Factors with Margins                    0
DQ ReliabilityScore of Factors without Margins                0
DQ TemporalCorrelation of Factors without Margins             0
DQ GeographicalCorrelation of Factors without Margins         0
DQ TechnologicalCorrelation of Factors without Margins        0
DQ DataCollection of Factors without Margins                  0
Source                                                        0
Year                                                          0
Industry Code                                             11032
Industry Name                           